# IA - Modèle MLP - BDD signaux acceptables

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import Fonctions_BDD_Signaux_Artificiels as BDD
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle

In [2]:
liste_comp = BDD.BDD_type('A')

liste_train = []
liste_test = []

for i in range(1,229):
    if i % 4 == 0 :
        liste_test.append(liste_comp[i-1])
    else :
        liste_train.append(liste_comp[i-1])

print("Liste complète", len(liste_comp))
print("75%", len(liste_train))
print("25%", len(liste_test))

Liste complète 228
75% 171
25% 57


### Définition des labels

In [3]:
target_train = []
target_test = []

for i in range(0,43):
    target_train.append('Gendarmerie')
for i in range(43,86):
    target_train.append('Police')    
for i in range(86,129):
    target_train.append('Pompiers')
for i in range(129,171):
    target_train.append('Samu')

for i in range(0,14):
    target_test.append('Gendarmerie')
for i in range(14,28):
    target_test.append('Police')
for i in range(28,42):
    target_test.append('Pompiers')
for i in range(42,57):
    target_test.append('Samu')

print(len(target_train))
print(len(target_test))

171
57


### Mélange de la BDD

In [4]:
liste_test_s, target_test_s = shuffle(liste_test, target_test)
liste_train_s, target_train_s = shuffle(liste_train, target_train)

### Définition de image_train et image_test

In [18]:
image_train = BDD.return_image(liste_train_s, 'C:/Users/abras/OneDrive - etu.sorbonne-universite.fr/M1/Projet/Projet_final/Sirenes artificielles')
image_test = BDD.return_image(liste_test_s, 'C:/Users/abras/OneDrive - etu.sorbonne-universite.fr/M1/Projet/Projet_final/Sirenes artificielles')

print(image_train.shape)
print(image_test.shape)

(171, 41, 27)
(57, 41, 27)


### Encodage des label_train et label_test

In [7]:
encodage = {"Gendarmerie":0, "Police":1, "Pompiers":2, "Samu":3}
sirenes_labels_numeriques_train = []
for label in target_train_s :
    sirenes_labels_numeriques_train.append(encodage[label])

sirenes_labels_numeriques_test = []
for label in target_test_s :
    sirenes_labels_numeriques_test.append(encodage[label])

In [8]:
labels_train_onehot = tf.keras.utils.to_categorical(sirenes_labels_numeriques_train)
labels_test_onehot = tf.keras.utils.to_categorical(sirenes_labels_numeriques_test)

print(labels_train_onehot.shape)
print(labels_test_onehot.shape)

(171, 4)
(57, 4)


### IA test

In [9]:
# On convertit nos valeurs en float

image_train = image_train.astype(float)
image_test = image_test.astype(float)

# On normalise les pixels pour avoir des valeurs entre 0 et 1

image_train = image_train/255
image_test = image_test/255

In [10]:
model = tf.keras.models.Sequential()

In [11]:
model.add(tf.keras.layers.Flatten(input_shape=[41,27]))

In [12]:
model.add(tf.keras.layers.Dense(400, activation="relu"))
model.add(tf.keras.layers.Dense(200, activation="relu"))
model.add(tf.keras.layers.Dense(4, activation="softmax"))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1107)              0         
_________________________________________________________________
dense (Dense)                (None, 400)               443200    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               80200     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 804       
Total params: 524,204
Trainable params: 524,204
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [15]:
history = model.fit(image_train,
                    labels_train_onehot,
                    epochs=10,
                    validation_split=0.1)

Epoch 1/10
5/5 [==============================] - 2s 265ms/step - loss: 92.4155 - accuracy: 0.2673 - val_loss: 22.4070 - val_accuracy: 0.4444
Epoch 2/10
5/5 [==============================] - 0s 44ms/step - loss: 21.8910 - accuracy: 0.5984 - val_loss: 30.8169 - val_accuracy: 0.2222
Epoch 3/10
5/5 [==============================] - 0s 43ms/step - loss: 11.6353 - accuracy: 0.5729 - val_loss: 6.7648 - val_accuracy: 0.6667
Epoch 4/10
5/5 [==============================] - 0s 42ms/step - loss: 4.3624 - accuracy: 0.7115 - val_loss: 5.1903 - val_accuracy: 0.6667
Epoch 5/10
5/5 [==============================] - 0s 43ms/step - loss: 0.6476 - accuracy: 0.9392 - val_loss: 2.7219 - val_accuracy: 0.7778
Epoch 6/10
5/5 [==============================] - 0s 41ms/step - loss: 0.1731 - accuracy: 0.9539 - val_loss: 3.9871 - val_accuracy: 0.7222
Epoch 7/10
5/5 [==============================] - 0s 48ms/step - loss: 0.0261 - accuracy: 0.9870 - val_loss: 3.4291 - val_accuracy: 0.6111
Epoch 8/10
5/5 [=====

In [16]:
loss_curve = history.history["loss"]
acc_curve = history.history["accuracy"]

loss_val_curve = history.history["val_loss"]
acc_val_curve = history.history["val_accuracy"]

plt.figure()
plt.plot(loss_curve, label="Train")
plt.plot(loss_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("Loss")
plt.show()

plt.figure()
plt.plot(acc_curve, label="Train")
plt.plot(acc_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("Accuracy")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
loss, acc = model.evaluate(image_test, labels_test_onehot)
print("Test Loss", loss)
print("Test Accuracy", acc)

2/2 [==============================] - 0s 7ms/step - loss: 2.9623 - accuracy: 0.8596
Test Loss 2.9623196125030518
Test Accuracy 0.859649121761322
